In [80]:
import os
import warnings
import numpy as np
import pandas as pd
import seaborn as sns
import geopandas as gpd
import matplotlib.pyplot as plt

from sklearn.linear_model import LinearRegression

warnings.filterwarnings(action='ignore')
plt.style.use('seaborn-whitegrid')
plt.rc('font', family='Malgun Gothic')  # 한글 폰트 사용을 위해서 세팅
pd.set_option('max_columns', 80) # 최대 컬럼 갯수 지정
pd.set_option('max_rows', 80) # 최대 로우 갯수 지정

In [81]:
def load_data(data):
    ### 데이터를 로드하여 전처리하는 함수 ###
    df = pd.read_csv(data, encoding='cp949')
    df = df[df['상권_구분_코드_명'] != '관광특구']
    df['상권_코드'] = df['상권_코드'].astype('str')
    df['상권_코드'] = df['상권_코드'].apply(lambda x: str(x))
    df['서비스_업종_코드_명'] = df['서비스_업종_코드_명'].apply(service_recategorization)
    return df

def service_recategorization(x):
    ### 서비스 업종별 코드 전처리 ###
    if x == '스포츠클럽' or x == '운동/경기용품' or x == '노래방' or x == 'PC방' or x == '골프연습장' or x == '당구장' or x == '스포츠 강습':
        return '여가'
    elif x == '편의점' or x == '슈퍼마켓' or x == '세탁소' or x == '부동산중개업':
        return '편의시설'
    elif x == '청과상' or x == '수산물판매' or x == '미곡판매' or x == '육류판매':
        return '농수산물'
    elif x == '자동차수리' or x == '자동차미용' or x == '자전거 및 기타운송장비':
        return '운송장비'
    elif x == '컴퓨터및주변장치판매' or x == '가전제품' or x == '가전제품수리' or x == '핸드폰':
        return '가전'
    elif x == '가방' or x == '신발' or x == '일반의류' or x == '안경' or x == '섬유제품':
        return '의류'
    elif x == '예술학원' or x == '외국어학원' or x == '일반교습학원' or x == '문구' or x == '서적' or x == '고시원' :
        return '교육'
    elif x == '화장품' or x == '시계및귀금속' or x == '피부관리실' or x == '네일숍' or x == '미용실':
        return '미용'
    elif x == '치과의원' or x == '일반의원' or x == '의료기기' or x == '한의원' or x == '의약품':
        return '의료'
    elif x == '전자상거래업' or x == '여관' or x == '완구' or x == '철물점' or x == '화초' or x == '애완동물':
        return '기타'
    else:
        return '음식'

In [82]:
store_2021 = load_data('서울시 우리마을가게 상권분석서비스(상권-추정매출).csv') 
store_2020 = load_data('서울시 우리마을가게상권분석서비스(상권-추정매출)_2020.csv') 
store_2019 = load_data('서울시 우리마을가게 상권분석서비스(상권-추정매출)_2019.csv')
store_2018 = load_data('서울시 우리마을가게 상권분석서비스(상권-추정매출)_2018.csv')

In [83]:
store_2021.head()

,기준_년_코드,기준_분기_코드,상권_구분_코드,상권_구분_코드_명,상권_코드,상권_코드_명,서비스_업종_코드,서비스_업종_코드_명,분기당_매출_금액,분기당_매출_건수,주중_매출_비율,주말_매출_비율,월요일_매출_비율,화요일_매출_비율,수요일_매출_비율,목요일_매출_비율,금요일_매출_비율,토요일_매출_비율,일요일_매출_비율,시간대_00~06_매출_비율,시간대_06~11_매출_비율,시간대_11~14_매출_비율,시간대_14~17_매출_비율,시간대_17~21_매출_비율,시간대_21~24_매출_비율,남성_매출_비율,여성_매출_비율,연령대_10_매출_비율,연령대_20_매출_비율,연령대_30_매출_비율,연령대_40_매출_비율,연령대_50_매출_비율,연령대_60_이상_매출_비율,주중_매출_금액,주말_매출_금액,월요일_매출_금액,화요일_매출_금액,수요일_매출_금액,목요일_매출_금액,금요일_매출_금액,토요일_매출_금액,일요일_매출_금액,시간대_00~06_매출_금액,시간대_06~11_매출_금액,시간대_11~14_매출_금액,시간대_14~17_매출_금액,시간대_17~21_매출_금액,시간대_21~24_매출_금액,남성_매출_금액,여성_매출_금액,연령대_10_매출_금액,연령대_20_매출_금액,연령대_30_매출_금액,연령대_40_매출_금액,연령대_50_매출_금액,연령대_60_이상_매출_금액,주중_매출_건수,주말_매출_건수,월요일_매출_건수,화요일_매출_건수,수요일_매출_건수,목요일_매출_건수,금요일_매출_건수,토요일_매출_건수,일요일_매출_건수,시간대_건수~06_매출_건수,시간대_건수~11_매출_건수,시간대_건수~14_매출_건수,시간대_건수~17_매출_건수,시간대_건수~21_매출_건수,시간대_건수~24_매출_건수,남성_매출_건수,여성_매출_건수,연령대_10_매출_건수,연령대_20_매출_건수,연령대_30_매출_건수,연령대_40_매출_건수,연령대_50_매출_건수,연령대_60_이상_매출_건수,점포수
277,2021,2,R,전통시장,1001490,길동복조리시장,CS300035,음식,38015377,2484,90,10,20,15,24,9,23,9,1,0,13,15,11,61,0,8,92,0,2,2,57,6,33,34381721,3633656,7744506,5706478,9060408,3281433,8588896,3261312,372344,0,4781536,5737630,4188226,23307985,0,2901036,35114341,0,905790,815211,21603094,2114468,12576814,2247,237,272,634,380,310,651,218,19,0,219,310,309,1646,0,472,2012,0,91,181,1356,256,600,4
278,2021,2,R,전통시장,1001490,길동복조리시장,CS300029,기타,8544694,460,75,25,14,9,16,16,20,25,0,0,8,29,30,33,0,30,71,0,6,15,21,25,33,6381424,2163270,1188807,774836,1343391,1357082,1717308,2163270,0,0,693577,2487664,2577626,2785827,0,2426241,5784461,0,473475,1229756,1736247,2077533,2693691,357,103,83,58,58,88,70,103,0,0,32,122,134,172,0,122,332,0,45,51,103,109,147,1
279,2021,2,R,전통시장,1001490,길동복조리시장,CS300028,기타,183304653,6800,58,42,4,8,8,16,23,28,14,0,8,17,21,46,8,59,41,3,39,35,10,12,2,106988169,76316484,7289932,14054148,14685008,29077017,41882064,51257340,25059144,0,14790152,31578026,37698765,84114604,15123106,99693332,68623499,4521159,64820818,58810130,16577587,20432839,3154298,3678,3122,385,596,630,665,1402,2175,947,0,560,1226,1401,3052,561,3645,2945,280,2628,2140,666,701,175,4
280,2021,2,R,전통시장,1001490,길동복조리시장,CS300027,의류,97268580,1573,57,43,11,14,19,9,4,23,21,0,6,22,37,35,0,34,66,0,0,9,9,33,50,55106108,42162472,10620059,14026152,18033017,8288228,4138652,22118193,20044279,0,6026216,21056732,36033977,34151655,0,33231459,63267745,0,0,8236603,8578075,31465294,48219232,1033,540,250,241,242,162,138,285,255,0,87,343,531,612,0,532,1029,0,0,197,238,402,723,6
281,2021,2,R,전통시장,1001490,길동복조리시장,CS300022,미용,749922143,39840,73,27,14,18,12,14,16,15,12,0,7,28,31,34,0,32,69,0,1,8,15,23,53,549739366,200182777,100879511,132108656,91636199,105353357,119761643,109395032,90787745,0,51615283,206811264,234245762,256401535,848299,236435984,513486159,0,10638327,60703321,111944392,170791310,395844793,30349,9491,5832,6804,4957,6579,6177,4602,4889,0,2709,9761,13360,13925,85,13549,26291,0,972,3671,4764,9461,20972,5


In [102]:
# 1496개의 상권코드를 가지고 있는 파일 로드
store_code = gpd.read_file('상권_geometry_좌표.geojson')

# store_code안에 관광특구 데이터 제거
store_code = store_code[store_code['TRDAR_SE_1']!='관광특구']

# store_code의 column 이름 재설정
store_code = store_code.rename(columns={'TRDAR_CD':'상권_코드', 'TRDAR_SE_1': '상권_구분_코드_명'})

# store_code 상권코드 str화
store_code['상권_코드'] = store_code['상권_코드'].astype('str')
store_code['상권_코드'] = store_code['상권_코드'].apply(lambda x: str(x))

# 데이터프레임 재구성
store_code = store_code[['상권_코드','상권_구분_코드_명']]
store_code.sort_values(by='상권_코드', inplace=True)

# 형태 확인
print(store_code.shape)
store_code.head()

(1490, 2)


,상권_코드,상권_구분_코드_명
914,1000001,골목상권
915,1000002,골목상권
916,1000003,골목상권
856,1000004,골목상권
857,1000005,골목상권


# 2021년

In [86]:
# merge를 할 새로운 테이블 생성
new_table = pd.DataFrame()

# 분기당 매출 금액을 기준으로 비율을 구하는 함수
def make_ratio(df,col):
    col_revenue = df.groupby([ '상권_코드', '기준_분기_코드'])[col].sum()
    quart_revenue = df.groupby(['상권_코드', '기준_분기_코드'])[ '분기당_매출_금액'].sum()
    tmp_df = (col_revenue / quart_revenue).to_frame()
    col = col.replace('_', ' ') # column의 이름에서 _를 띄어쓰기로 변환
    tmp_df.columns = [col]
    
    return tmp_df

In [87]:
# 여성 총매출 비율
female = make_ratio(store, '여성_매출_금액')
new_table = female
new_table

여성 매출 금액
상권_코드   기준_분기_코드          
1000001 1         0.536839
        2         0.550910
        3         0.519923
        4         0.517027
1000002 1         0.539067
...                    ...
1001489 4         0.491056
1001490 1         0.594786
        2         0.534639
        3         0.591459
        4         0.559165

[5917 rows x 1 columns]

In [88]:
# 남성 총매출 비율
male = make_ratio(store, '남성_매출_금액')
new_table = pd.merge(new_table, male, right_index=True, left_index=True)
new_table

여성 매출 금액  남성 매출 금액
상권_코드   기준_분기_코드                    
1000001 1         0.536839  0.401229
        2         0.550910  0.388514
        3         0.519923  0.390261
        4         0.517027  0.392987
1000002 1         0.539067  0.423041
...                    ...       ...
1001489 4         0.491056  0.467931
1001490 1         0.594786  0.373079
        2         0.534639  0.453883
        3         0.591459  0.391917
        4         0.559165  0.403125

[5917 rows x 2 columns]

In [89]:
# 연령대별 총매출 비율
ages = ['연령대_10_매출_금액', '연령대_20_매출_금액', '연령대_30_매출_금액',
        '연령대_40_매출_금액', '연령대_50_매출_금액', '연령대_60_이상_매출_금액']

for age in ages:
    age = make_ratio(store, age)
    new_table = pd.merge(new_table, age, right_index=True, left_index=True)
    
new_table

여성 매출 금액  남성 매출 금액  연령대 10 매출 금액  연령대 20 매출 금액  \
상권_코드   기준_분기_코드                                                   
1000001 1         0.536839  0.401229      0.004629      0.089526   
        2         0.550910  0.388514      0.008672      0.159500   
        3         0.519923  0.390261      0.011274      0.173040   
        4         0.517027  0.392987      0.011346      0.192749   
1000002 1         0.539067  0.423041      0.004235      0.186656   
...                    ...       ...           ...           ...   
1001489 4         0.491056  0.467931      0.006687      0.103577   
1001490 1         0.594786  0.373079      0.000626      0.056362   
        2         0.534639  0.453883      0.000518      0.039673   
        3         0.591459  0.391917      0.000582      0.058256   
        4         0.559165  0.403125      0.000970      0.043611   

                  연령대 30 매출 금액  연령대 40 매출 금액  연령대 50 매출 금액  연령대 60 이상 매출 금액  
상권_코드   기준_분기_코드                                                             
1000001 1             0.168035      0.258008      0.164013         0.253856  
        2             0.179625      0.205077      0.223253         0.163298  
        3             0.202655      0.225887      0.174055         0.123272  
        4             0.219065      0.182077      0.186077         0.118700  
1000002 1             0.176573      0.204849      0.203784         0.186011  
...                        ...           ...           ...              ...  
1001489 4             0.139988      0.199599      0.273256         0.235880  
1001490 1             0.092622      0.171802      0.288095         0.358358  
        2             0.111816      0.161141      0.304551         0.370824  
        3             0.139521      0.170457      0.284514         0.330047  
        4             0.103765      0.181804      0.303605         0.328535  

[5917 rows x 8 columns]

In [90]:
# 시간대별 총매출 비율
time_for_prepro = store_2021[['상권_코드', '기준_년_코드', '기준_분기_코드', '분기당_매출_금액','시간대_00~06_매출_비율',
                              '시간대_06~11_매출_비율', '시간대_11~14_매출_비율', '시간대_14~17_매출_비율',
                              '시간대_17~21_매출_비율', '시간대_21~24_매출_비율']]

# 비율 소수점 화
time_for_prepro.iloc[:,-4:] = time_for_prepro.iloc[:,-4:].apply(lambda x: x * 0.01)

# 시간대별 총매출액 (시간대별은 건수와 비율밖에 존재하지 않음)
times_revenue = ['시간대_00~06_매출액', '시간대_06~11_매출액', '시간대_11~14_매출액', 
                 '시간대_14~17_매출액', '시간대_17~21_매출액', '시간대_21~24_매출액']
for time_revenue, time in zip(times_revenue, times):
    time_for_prepro[time_revenue] = list(map(int,(time_for_prepro.loc[:,time] * time_for_prepro.loc[:,'분기당_매출_금액'])))
    time_for_prepro=time_for_prepro.drop(time, axis=1)

# 계산된 시간대별 총매출액을 기준으로 총매출액 비율 계산
for time_revenue in times_revenue:
    time_revenue = make_ratio(time_for_prepro, time_revenue)
    new_table = pd.merge(new_table, time_revenue, right_index=True, left_index=True)

new_table.head()

여성 매출 금액  남성 매출 금액  연령대 10 매출 금액  연령대 20 매출 금액  \
상권_코드   기준_분기_코드                                                   
1000001 1         0.536839  0.401229      0.004629      0.089526   
        2         0.550910  0.388514      0.008672      0.159500   
1000002 1         0.539067  0.423041      0.004235      0.186656   
        2         0.497391  0.478170      0.002083      0.136804   
1000003 1         0.372600  0.542733      0.002199      0.212092   

                  연령대 30 매출 금액  연령대 40 매출 금액  연령대 50 매출 금액  연령대 60 이상 매출 금액  \
상권_코드   기준_분기_코드                                                              
1000001 1             0.168035      0.258008      0.164013         0.253856   
        2             0.179625      0.205077      0.223253         0.163298   
1000002 1             0.176573      0.204849      0.203784         0.186011   
        2             0.176321      0.225849      0.227031         0.207473   
1000003 1             0.292881      0.166224      0.139362         0.102576   

                  시간대 00~06 매출액  시간대 06~11 매출액  시간대 11~14 매출액  시간대 14~17 매출액  \
상권_코드   기준_분기_코드                                                               
1000001 1              0.835285       6.839895       0.303365       0.303549   
        2              0.556968       6.152917       0.325921       0.285469   
1000002 1              1.803000       6.030559       0.272554       0.357113   
        2              1.258649       6.193667       0.281284       0.342988   
1000003 1              1.712387       4.860777       0.253126       0.358296   

                  시간대 17~21 매출액  시간대 21~24 매출액  
상권_코드   기준_분기_코드                                
1000001 1              0.275303       0.045415  
        2              0.278418       0.043466  
1000002 1              0.242619       0.048048  
        2              0.272574       0.037734  
1000003 1              0.265619       0.059849

In [91]:
# 연령대별 총매출 비율
total_cat_revenue = store_2021.groupby(['상권_코드', '기준_분기_코드', '서비스_업종_코드_명'])['분기당_매출_금액'].sum()
quat_ratio_revenue = store_2021.groupby(['상권_코드','기준_분기_코드'])['분기당_매출_금액'].sum()
tmp_df = (total_cat_revenue / quat_ratio_revenue).unstack(level=-1)
new_table = pd.merge(new_table, tmp_df, right_index=True, left_index=True)

new_table.head()

여성 매출 금액  남성 매출 금액  연령대 10 매출 금액  연령대 20 매출 금액  \
상권_코드   기준_분기_코드                                                   
1000001 1         0.536839  0.401229      0.004629      0.089526   
        2         0.550910  0.388514      0.008672      0.159500   
1000002 1         0.539067  0.423041      0.004235      0.186656   
        2         0.497391  0.478170      0.002083      0.136804   
1000003 1         0.372600  0.542733      0.002199      0.212092   

                  연령대 30 매출 금액  연령대 40 매출 금액  연령대 50 매출 금액  연령대 60 이상 매출 금액  \
상권_코드   기준_분기_코드                                                              
1000001 1             0.168035      0.258008      0.164013         0.253856   
        2             0.179625      0.205077      0.223253         0.163298   
1000002 1             0.176573      0.204849      0.203784         0.186011   
        2             0.176321      0.225849      0.227031         0.207473   
1000003 1             0.292881      0.166224      0.139362         0.102576   

                  시간대 00~06 매출액  시간대 06~11 매출액  시간대 11~14 매출액  시간대 14~17 매출액  \
상권_코드   기준_분기_코드                                                               
1000001 1              0.835285       6.839895       0.303365       0.303549   
        2              0.556968       6.152917       0.325921       0.285469   
1000002 1              1.803000       6.030559       0.272554       0.357113   
        2              1.258649       6.193667       0.281284       0.342988   
1000003 1              1.712387       4.860777       0.253126       0.358296   

                  시간대 17~21 매출액  시간대 21~24 매출액        가전        교육        기타  \
상권_코드   기준_분기_코드                                                               
1000001 1              0.275303       0.045415       NaN  0.036231       NaN   
        2              0.278418       0.043466       NaN  0.025179       NaN   
1000002 1              0.242619       0.048048       NaN       NaN  0.001418   
        2              0.272574       0.037734       NaN       NaN  0.000948   
1000003 1              0.265619       0.059849  0.007205  0.008727  0.096980   

                     농수산물        미용        여가  운송장비        음식        의료  \
상권_코드   기준_분기_코드                                                          
1000001 1             NaN  0.105437       NaN   NaN  0.529584  0.011414   
        2             NaN  0.082550       NaN   NaN  0.591812  0.004702   
1000002 1             NaN  0.007136  0.005744   NaN  0.081401       NaN   
        2             NaN  0.001573  0.004352   NaN  0.066194       NaN   
1000003 1         0.00003  0.493331  0.000947   NaN  0.268324       NaN   

                        의류      편의시설  
상권_코드   기준_분기_코드                      
1000001 1         0.088071  0.229263  
        2         0.085216  0.210540  
1000002 1         0.729628  0.174673  
        2         0.801042  0.125891  
1000003 1         0.000059  0.124397

In [93]:
new_table = new_table.fillna(-1)
new_table = new_table.unstack()
new_table.columns = ['2021년 '+str(j) +  ' 분기 ' + str(i) for i, j in new_table.columns]
print(new_table.shape)
new_table.head()

(1479, 50)


,2021년 1 분기 여성 매출 금액,2021년 2 분기 여성 매출 금액,2021년 1 분기 남성 매출 금액,2021년 2 분기 남성 매출 금액,2021년 1 분기 연령대 10 매출 금액,2021년 2 분기 연령대 10 매출 금액,2021년 1 분기 연령대 20 매출 금액,2021년 2 분기 연령대 20 매출 금액,2021년 1 분기 연령대 30 매출 금액,2021년 2 분기 연령대 30 매출 금액,2021년 1 분기 연령대 40 매출 금액,2021년 2 분기 연령대 40 매출 금액,2021년 1 분기 연령대 50 매출 금액,2021년 2 분기 연령대 50 매출 금액,2021년 1 분기 연령대 60 이상 매출 금액,2021년 2 분기 연령대 60 이상 매출 금액,2021년 1 분기 시간대 00~06 매출액,2021년 2 분기 시간대 00~06 매출액,2021년 1 분기 시간대 06~11 매출액,2021년 2 분기 시간대 06~11 매출액,2021년 1 분기 시간대 11~14 매출액,2021년 2 분기 시간대 11~14 매출액,2021년 1 분기 시간대 14~17 매출액,2021년 2 분기 시간대 14~17 매출액,2021년 1 분기 시간대 17~21 매출액,2021년 2 분기 시간대 17~21 매출액,2021년 1 분기 시간대 21~24 매출액,2021년 2 분기 시간대 21~24 매출액,2021년 1 분기 가전,2021년 2 분기 가전,2021년 1 분기 교육,2021년 2 분기 교육,2021년 1 분기 기타,2021년 2 분기 기타,2021년 1 분기 농수산물,2021년 2 분기 농수산물,2021년 1 분기 미용,2021년 2 분기 미용,2021년 1 분기 여가,2021년 2 분기 여가,2021년 1 분기 운송장비,2021년 2 분기 운송장비,2021년 1 분기 음식,2021년 2 분기 음식,2021년 1 분기 의료,2021년 2 분기 의료,2021년 1 분기 의류,2021년 2 분기 의류,2021년 1 분기 편의시설,2021년 2 분기 편의시설
상권_코드,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1000001,0.536839,0.550910,0.401229,0.388514,0.004629,0.008672,0.089526,0.159500,0.168035,0.179625,0.258008,0.205077,0.164013,0.223253,0.253856,0.163298,0.835285,0.556968,6.839895,6.152917,0.303365,0.325921,0.303549,0.285469,0.275303,0.278418,0.045415,0.043466,-1.000000,-1.000000,0.036231,0.025179,-1.000000,-1.000000,-1.000000,-1.000000,0.105437,0.082550,-1.000000,-1.000000,-1.0,-1.0,0.529584,0.591812,0.011414,0.004702,0.088071,0.085216,0.229263,0.210540
1000002,0.539067,0.497391,0.423041,0.478170,0.004235,0.002083,0.186656,0.136804,0.176573,0.176321,0.204849,0.225849,0.203784,0.227031,0.186011,0.207473,1.803000,1.258649,6.030559,6.193667,0.272554,0.281284,0.357113,0.342988,0.242619,0.272574,0.048048,0.037734,-1.000000,-1.000000,-1.000000,-1.000000,0.001418,0.000948,-1.000000,-1.000000,0.007136,0.001573,0.005744,0.004352,-1.0,-1.0,0.081401,0.066194,-1.000000,-1.000000,0.729628,0.801042,0.174673,0.125891
1000003,0.372600,0.373874,0.542733,0.531110,0.002199,0.003525,0.212092,0.239427,0.292881,0.258305,0.166224,0.156257,0.139362,0.145764,0.102576,0.101706,1.712387,1.837665,4.860777,5.180167,0.253126,0.276004,0.358296,0.289483,0.265619,0.269148,0.059849,0.095262,0.007205,0.005451,0.008727,0.011363,0.096980,0.103175,0.000030,-1.000000,0.493331,0.412665,0.000947,0.002410,-1.0,-1.0,0.268324,0.318861,-1.000000,-1.000000,0.000059,0.007307,0.124397,0.138768
1000004,0.411807,0.366163,0.577116,0.620050,0.010928,0.010321,0.353791,0.322687,0.123618,0.165043,0.139339,0.140690,0.211079,0.222165,0.150169,0.125308,3.630118,3.591929,9.270843,9.135487,0.253818,0.263913,0.198561,0.218451,0.308543,0.271482,0.116830,0.123938,0.005453,0.100524,-1.000000,-1.000000,-1.000000,-1.000000,0.015544,0.016927,-1.000000,-1.000000,-1.000000,-1.000000,-1.0,-1.0,0.376782,0.380902,0.139269,0.103460,-1.000000,-1.000000,0.462952,0.398187
1000005,0.413092,0.460960,0.500522,0.438834,0.008816,0.004659,0.156688,0.165914,0.169615,0.196526,0.226904,0.226223,0.184040,0.207485,0.167552,0.098987,0.287592,0.204652,1.146796,1.294579,0.184802,0.205705,0.257101,0.241196,0.464797,0.439395,0.077966,0.099300,-1.000000,-1.000000,0.093592,0.044426,0.007068,0.010790,-1.000000,-1.000000,0.001242,0.001302,-1.000000,-1.000000,-1.0,-1.0,0.791146,0.842230,-1.000000,-1.000000,-1.000000,0.005450,0.106952,0.095803


# 2020년

In [15]:
# 여성 총매출 비율
female = make_ratio(store_2020, '여성_매출_금액')
new_table_2020 = female

# 남성 총매출 비율
male = make_ratio(store_2020, '남성_매출_금액')
new_table_2020 = pd.merge(new_table_2020, male, right_index=True, left_index=True)

# 연령대별 총매출 비율
ages = ['연령대_10_매출_금액', '연령대_20_매출_금액', '연령대_30_매출_금액',
        '연령대_40_매출_금액', '연령대_50_매출_금액', '연령대_60_이상_매출_금액']
for age in ages:
    age = make_ratio(store, age)
    new_table_2020 = pd.merge(new_table_2020, age, right_index=True, left_index=True)
    
# 시간대별 총매출 비율
time_for_prepro = store_2020[['상권_코드', '기준_년_코드', '기준_분기_코드', '분기당_매출_금액','시간대_00~06_매출_비율',
                              '시간대_06~11_매출_비율', '시간대_11~14_매출_비율', '시간대_14~17_매출_비율',
                              '시간대_17~21_매출_비율', '시간대_21~24_매출_비율']]

# 비율 소수점 화
times = ['시간대_00~06_매출_비율', '시간대_06~11_매출_비율', '시간대_11~14_매출_비율', 
         '시간대_14~17_매출_비율', '시간대_17~21_매출_비율', '시간대_21~24_매출_비율']

time_for_prepro.iloc[:,-4:] = time_for_prepro.iloc[:,-4:].apply(lambda x: x * 0.01)

# 시간대별 총매출액 (시간대별은 건수와 비율밖에 존재하지 않음)
times_revenue = ['시간대_00~06_매출액', '시간대_06~11_매출액', '시간대_11~14_매출액', 
                 '시간대_14~17_매출액', '시간대_17~21_매출액', '시간대_21~24_매출액']
for time_revenue, time in zip(times_revenue, times):
    time_for_prepro[time_revenue] = list(map(int,(time_for_prepro.loc[:,time] * time_for_prepro.loc[:,'분기당_매출_금액'])))
    time_for_prepro=time_for_prepro.drop(time, axis=1)

# 계산된 시간대별 총매출액을 기준으로 총매출액 비율 계산
for time_revenue in times_revenue:
    time_revenue = make_ratio(time_for_prepro, time_revenue)
    new_table_2020 = pd.merge(new_table_2020, time_revenue, right_index=True, left_index=True)

# 연령대별 총매출 비율
total_categoiesy_revenue = store.groupby(['상권_코드', '기준_분기_코드', '서비스_업종_코드_명'])['분기당_매출_금액'].sum()
quarter_ratio_revenue = store.groupby(['상권_코드','기준_분기_코드'])['분기당_매출_금액'].sum()
tmp_df = (total_categoiesy_revenue / quarter_ratio_revenue).unstack(level=-1)
new_table_2020 = pd.merge(new_table_2020, tmp_df, right_index=True, left_index=True)

new_table_2020 = new_table_2020.fillna(-1)
new_table_2020 = new_table_2020.unstack()
new_table_2020.columns = ['2020년 '+str(j) +  ' 분기 ' + str(i) for i, j in new_table_2020.columns]
print(new_table_2020.shape)
new_table_2020.head()

pd.merge(new_table, new_table_2020, right_index=True, left_index=True)

(1480, 100)


,2021년 1 분기 여성 매출 금액,2021년 2 분기 여성 매출 금액,2021년 1 분기 남성 매출 금액,2021년 2 분기 남성 매출 금액,2021년 1 분기 연령대 10 매출 금액,2021년 2 분기 연령대 10 매출 금액,2021년 1 분기 연령대 20 매출 금액,2021년 2 분기 연령대 20 매출 금액,2021년 1 분기 연령대 30 매출 금액,2021년 2 분기 연령대 30 매출 금액,2021년 1 분기 연령대 40 매출 금액,2021년 2 분기 연령대 40 매출 금액,2021년 1 분기 연령대 50 매출 금액,2021년 2 분기 연령대 50 매출 금액,2021년 1 분기 연령대 60 이상 매출 금액,2021년 2 분기 연령대 60 이상 매출 금액,2021년 1 분기 시간대 00~06 매출액,2021년 2 분기 시간대 00~06 매출액,2021년 1 분기 시간대 06~11 매출액,2021년 2 분기 시간대 06~11 매출액,2021년 1 분기 시간대 11~14 매출액,2021년 2 분기 시간대 11~14 매출액,2021년 1 분기 시간대 14~17 매출액,2021년 2 분기 시간대 14~17 매출액,2021년 1 분기 시간대 17~21 매출액,2021년 2 분기 시간대 17~21 매출액,2021년 1 분기 시간대 21~24 매출액,2021년 2 분기 시간대 21~24 매출액,2021년 1 분기 가전,2021년 2 분기 가전,2021년 1 분기 교육,2021년 2 분기 교육,2021년 1 분기 기타,2021년 2 분기 기타,2021년 1 분기 농수산물,2021년 2 분기 농수산물,2021년 1 분기 미용,2021년 2 분기 미용,2021년 1 분기 여가,2021년 2 분기 여가,...,2020년 1 분기 교육,2020년 2 분기 교육,2020년 3 분기 교육,2020년 4 분기 교육,2020년 1 분기 기타,2020년 2 분기 기타,2020년 3 분기 기타,2020년 4 분기 기타,2020년 1 분기 농수산물,2020년 2 분기 농수산물,2020년 3 분기 농수산물,2020년 4 분기 농수산물,2020년 1 분기 미용,2020년 2 분기 미용,2020년 3 분기 미용,2020년 4 분기 미용,2020년 1 분기 여가,2020년 2 분기 여가,2020년 3 분기 여가,2020년 4 분기 여가,2020년 1 분기 운송장비,2020년 2 분기 운송장비,2020년 3 분기 운송장비,2020년 4 분기 운송장비,2020년 1 분기 음식,2020년 2 분기 음식,2020년 3 분기 음식,2020년 4 분기 음식,2020년 1 분기 의료,2020년 2 분기 의료,2020년 3 분기 의료,2020년 4 분기 의료,2020년 1 분기 의류,2020년 2 분기 의류,2020년 3 분기 의류,2020년 4 분기 의류,2020년 1 분기 편의시설,2020년 2 분기 편의시설,2020년 3 분기 편의시설,2020년 4 분기 편의시설
상권_코드,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1000001,0.567886,0.585425,0.329257,0.303938,0.006921,0.008499,0.180985,0.185540,0.198078,0.229166,0.198989,0.199314,0.161625,0.153722,0.150544,0.113122,0.008353,0.005570,0.068399,0.061529,0.303365,0.325921,0.303549,0.285469,0.275303,0.278418,0.045415,0.043466,0.000000,0.000000,0.036231,0.025179,0.000000,0.000000,0.000000,0.000000,0.105437,0.082550,0.000000,0.000000,...,0.028444,0.037253,0.040966,0.038361,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.081214,0.109474,0.067002,0.045457,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.697263,0.542525,0.510781,0.524697,0.009033,0.006357,0.010409,0.010668,0.037438,0.100237,0.082397,0.115796,0.146607,0.204154,0.288445,0.265023
1000002,0.578683,0.639484,0.407471,0.353526,0.002219,0.002208,0.122634,0.107609,0.178571,0.140956,0.200006,0.219398,0.249588,0.282353,0.233137,0.240485,0.018030,0.012586,0.060306,0.061937,0.272554,0.281284,0.357113,0.342988,0.242619,0.272574,0.048048,0.037734,0.000000,0.000000,0.000000,0.000000,0.001418,0.000948,0.000000,0.000000,0.007136,0.001573,0.005744,0.004352,...,0.000000,0.000000,0.000000,0.000000,0.020094,0.010281,0.008349,0.005745,0.000000,0.000000,0.000000,0.000000,0.020542,0.012777,0.009200,0.005225,0.010148,0.009504,0.007484,0.011162,0.000000,0.000000,0.000000,0.000000,0.184060,0.129724,0.152184,0.080122,0.000000,0.000000,0.000000,0.000000,0.576600,0.686995,0.624865,0.763133,0.188555,0.141159,0.183996,0.134613
1000003,0.386436,0.428179,0.511372,0.474744,0.001972,0.004856,0.216906,0.213067,0.291503,0.261775,0.159254,0.155604,0.132897,0.163318,0.095275,0.104303,0.017124,0.018377,0.048608,0.051802,0.253126,0.276004,0.358296,0.289483,0.265619,0.269148,0.059849,0.095262,0.007205,0.005451,0.008727,0.011363,0.096980,0.103175,0.000030,0.000000,0.493331,0.412665,0.000947,0.002410,...,0.011599,0.015511,0.006986,0.007064,0.147192,0.107543,0.103492,0.113976,0.001365,0.001226,0.001140,0.000660,0.366426,0.342450,0.364773,0.386178,0.000141,0.000103,0.002191,0.001829,0.000000,0.000000,0.000000,0.000000,0.292328,0.354712,0.338810,0.317269,0.000000,0.000000,0.000000,0.000000,0.002202,0.004294,0.003422,0.004886,0.174096,0.166198,0.166720,0.149225
1000004,0.456745,0.430480,0.523239,0.496997,0.009383,0.013992,0.321122,0.288958,0.134135,0.129082,0.157252,0.154217,0.211101,0.190894,0.146991,0.150334,0.036301,0.035919,0.092708,0.091355,0.253818,0.263913,0.198561,0.218451,0.308543,0.271482,0.116830,0.123938,0.005453,0.100524,0.00

# 2019년

In [98]:
# 여성 총매출 비율
female = make_ratio(store_2019, '여성_매출_금액')
new_table_2019 = female

# 남성 총매출 비율
male = make_ratio(store_2019, '남성_매출_금액')
new_table_2019 = pd.merge(new_table_2019, male, right_index=True, left_index=True)

# 연령대별 총매출 비율
ages = ['연령대_10_매출_금액', '연령대_20_매출_금액', '연령대_30_매출_금액',
        '연령대_40_매출_금액', '연령대_50_매출_금액', '연령대_60_이상_매출_금액']
for age in ages:
    age = make_ratio(store_2019, age)
    new_table_2019 = pd.merge(new_table_2019, age, right_index=True, left_index=True)
    
# 시간대별 총매출 비율
time_for_prepro = store_2019[['상권_코드', '기준_년_코드', '기준_분기_코드', '분기당_매출_금액','시간대_00~06_매출_비율',
                              '시간대_06~11_매출_비율', '시간대_11~14_매출_비율', '시간대_14~17_매출_비율',
                              '시간대_17~21_매출_비율', '시간대_21~24_매출_비율']]

# 비율 소수점 화
times = ['시간대_00~06_매출_비율', '시간대_06~11_매출_비율', '시간대_11~14_매출_비율', 
         '시간대_14~17_매출_비율', '시간대_17~21_매출_비율', '시간대_21~24_매출_비율']

time_for_prepro.iloc[:,-4:] = time_for_prepro.iloc[:,-4:].apply(lambda x: x * 0.01)

# 시간대별 총매출액 (시간대별은 건수와 비율밖에 존재하지 않음)
times_revenue = ['시간대_00~06_매출액', '시간대_06~11_매출액', '시간대_11~14_매출액', 
                 '시간대_14~17_매출액', '시간대_17~21_매출액', '시간대_21~24_매출액']
for time_revenue, time in zip(times_revenue, times):
    time_for_prepro[time_revenue] = list(map(int,(time_for_prepro.loc[:,time] * time_for_prepro.loc[:,'분기당_매출_금액'])))
    time_for_prepro=time_for_prepro.drop(time, axis=1)

# 계산된 시간대별 총매출액을 기준으로 총매출액 비율 계산
for time_revenue in times_revenue:
    time_revenue = make_ratio(time_for_prepro, time_revenue)
    new_table_2019 = pd.merge(new_table_2019, time_revenue, right_index=True, left_index=True)

# 연령대별 총매출 비율
total_categoiesy_revenue = store_2019.groupby(['상권_코드', '기준_분기_코드', '서비스_업종_코드_명'])['분기당_매출_금액'].sum()
quarter_ratio_revenue = store_2019.groupby(['상권_코드','기준_분기_코드'])['분기당_매출_금액'].sum()
tmp_df = (total_categoiesy_revenue / quarter_ratio_revenue).unstack(level=-1)
new_table_2019 = pd.merge(new_table_2019, tmp_df, right_index=True, left_index=True)

new_table_2019 = new_table_2019.fillna(-1)
new_table_2019 = new_table_2019.unstack()
new_table_2019.columns = ['2019년 '+str(j) +  ' 분기 ' + str(i) for i, j in new_table_2019.columns]
print(new_table_2019.shape)

new_table = pd.merge(new_table, new_table_2019, right_index=True, left_index=True)
new_table

(1482, 100)


,2021년 1 분기 여성 매출 금액,2021년 2 분기 여성 매출 금액,2021년 1 분기 남성 매출 금액,2021년 2 분기 남성 매출 금액,2021년 1 분기 연령대 10 매출 금액,2021년 2 분기 연령대 10 매출 금액,2021년 1 분기 연령대 20 매출 금액,2021년 2 분기 연령대 20 매출 금액,2021년 1 분기 연령대 30 매출 금액,2021년 2 분기 연령대 30 매출 금액,2021년 1 분기 연령대 40 매출 금액,2021년 2 분기 연령대 40 매출 금액,2021년 1 분기 연령대 50 매출 금액,2021년 2 분기 연령대 50 매출 금액,2021년 1 분기 연령대 60 이상 매출 금액,2021년 2 분기 연령대 60 이상 매출 금액,2021년 1 분기 시간대 00~06 매출액,2021년 2 분기 시간대 00~06 매출액,2021년 1 분기 시간대 06~11 매출액,2021년 2 분기 시간대 06~11 매출액,2021년 1 분기 시간대 11~14 매출액,2021년 2 분기 시간대 11~14 매출액,2021년 1 분기 시간대 14~17 매출액,2021년 2 분기 시간대 14~17 매출액,2021년 1 분기 시간대 17~21 매출액,2021년 2 분기 시간대 17~21 매출액,2021년 1 분기 시간대 21~24 매출액,2021년 2 분기 시간대 21~24 매출액,2021년 1 분기 가전,2021년 2 분기 가전,2021년 1 분기 교육,2021년 2 분기 교육,2021년 1 분기 기타,2021년 2 분기 기타,2021년 1 분기 농수산물,2021년 2 분기 농수산물,2021년 1 분기 미용,2021년 2 분기 미용,2021년 1 분기 여가,2021년 2 분기 여가,...,2019년 1 분기 교육,2019년 2 분기 교육,2019년 3 분기 교육,2019년 4 분기 교육,2019년 1 분기 기타,2019년 2 분기 기타,2019년 3 분기 기타,2019년 4 분기 기타,2019년 1 분기 농수산물,2019년 2 분기 농수산물,2019년 3 분기 농수산물,2019년 4 분기 농수산물,2019년 1 분기 미용,2019년 2 분기 미용,2019년 3 분기 미용,2019년 4 분기 미용,2019년 1 분기 여가,2019년 2 분기 여가,2019년 3 분기 여가,2019년 4 분기 여가,2019년 1 분기 운송장비,2019년 2 분기 운송장비,2019년 3 분기 운송장비,2019년 4 분기 운송장비,2019년 1 분기 음식,2019년 2 분기 음식,2019년 3 분기 음식,2019년 4 분기 음식,2019년 1 분기 의료,2019년 2 분기 의료,2019년 3 분기 의료,2019년 4 분기 의료,2019년 1 분기 의류,2019년 2 분기 의류,2019년 3 분기 의류,2019년 4 분기 의류,2019년 1 분기 편의시설,2019년 2 분기 편의시설,2019년 3 분기 편의시설,2019년 4 분기 편의시설
상권_코드,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1000001,0.536839,0.550910,0.401229,0.388514,0.004629,0.008672,0.089526,0.159500,0.168035,0.179625,0.258008,0.205077,0.164013,0.223253,0.253856,0.163298,0.835285,0.556968,6.839895,6.152917,0.303365,0.325921,0.303549,0.285469,0.275303,0.278418,0.045415,0.043466,-1.000000,-1.000000,0.036231,0.025179,-1.000000,-1.000000,-1.000000,-1.000000,0.105437,0.082550,-1.000000,-1.000000,...,0.007638,0.009556,0.007834,0.027255,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,0.019189,0.019095,0.021578,0.049696,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,0.596421,0.555468,0.485714,0.654346,0.008164,0.005401,0.017975,0.008546,0.114989,0.163847,0.184980,0.107809,0.253599,0.246634,0.281918,0.152348
1000002,0.539067,0.497391,0.423041,0.478170,0.004235,0.002083,0.186656,0.136804,0.176573,0.176321,0.204849,0.225849,0.203784,0.227031,0.186011,0.207473,1.803000,1.258649,6.030559,6.193667,0.272554,0.281284,0.357113,0.342988,0.242619,0.272574,0.048048,0.037734,-1.000000,-1.000000,-1.000000,-1.000000,0.001418,0.000948,-1.000000,-1.000000,0.007136,0.001573,0.005744,0.004352,...,0.017445,0.013408,0.006766,0.013562,0.307695,0.090061,0.024097,0.024495,-1.000000,-1.000000,-1.000000,-1.000000,0.000979,0.001445,0.001046,0.002225,0.008891,0.011891,0.005054,0.008917,-1.000000,-1.000000,-1.000000,-1.000000,0.131770,0.143218,0.153742,0.153169,0.051721,0.077721,0.076658,-1.000000,0.323309,0.461968,0.559004,0.684968,0.157582,0.187658,0.173632,0.112664
1000003,0.372600,0.373874,0.542733,0.531110,0.002199,0.003525,0.212092,0.239427,0.292881,0.258305,0.166224,0.156257,0.139362,0.145764,0.102576,0.101706,1.712387,1.837665,4.860777,5.180167,0.253126,0.276004,0.358296,0.289483,0.265619,0.269148,0.059849,0.095262,0.007205,0.005451,0.008727,0.011363,0.096980,0.103175,0.000030,-1.000000,0.493331,0.412665,0.000947,0.002410,...,0.011549,0.020722,0.011639,0.009825,0.190430,0.152932,0.144465,0.161529,0.016115,0.016849,0.013192,0.003162,0.283561,0.287326,0.311214,0.319739,0.000247,0.000008,0.000160,0.000124,-1.000000,-1.000000,-1.000000,-1.000000,0.383422,0.400506,0.371634,0.335685,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,0.004043,0.112200,0.116105,0.142387,0.157031
1000004,0.411807,0.366163,0.577116,0.620050,0.010928,0.010321,0.353791,0.322687,0.123618,0.165043,0.139339,0.140690,0.211079,0.222165,0.150169,0.125308,3.630118,3.591929,9.270843,9.135487,0.253818,0.263913,0.198561,0.218451,0.30854

# 2018년

In [99]:
# 여성 총매출 비율
female = make_ratio(store_2018, '여성_매출_금액')
new_table_2018 = female

# 남성 총매출 비율
male = make_ratio(store_2018, '남성_매출_금액')
new_table_2018 = pd.merge(new_table_2018, male, right_index=True, left_index=True)

# 연령대별 총매출 비율
ages = ['연령대_10_매출_금액', '연령대_20_매출_금액', '연령대_30_매출_금액',
        '연령대_40_매출_금액', '연령대_50_매출_금액', '연령대_60_이상_매출_금액']
for age in ages:
    age = make_ratio(store_2018, age)
    new_table_2018 = pd.merge(new_table_2018, age, right_index=True, left_index=True)

# 시간대별 총매출 비율
time_for_prepro = store_2018[['상권_코드', '기준_년_코드', '기준_분기_코드', '분기당_매출_금액','시간대_00~06_매출_비율',
                              '시간대_06~11_매출_비율', '시간대_11~14_매출_비율', '시간대_14~17_매출_비율',
                              '시간대_17~21_매출_비율', '시간대_21~24_매출_비율']]

# 비율 소수점 화
times = ['시간대_00~06_매출_비율', '시간대_06~11_매출_비율', '시간대_11~14_매출_비율', 
         '시간대_14~17_매출_비율', '시간대_17~21_매출_비율', '시간대_21~24_매출_비율']

for time in times:
    time_for_prepro[time] = time_for_prepro[time] * 0.01

# 시간대별 총매출액 (시간대별은 건수와 비율밖에 존재하지 않음)
times_revenue = ['시간대_00~06_매출액', '시간대_06~11_매출액', '시간대_11~14_매출액', 
                 '시간대_14~17_매출액', '시간대_17~21_매출액', '시간대_21~24_매출액']
for time_revenue, time in zip(times_revenue, times):
    time_for_prepro[time_revenue] = list(map(int,(time_for_prepro.loc[:,time] * time_for_prepro.loc[:,'분기당_매출_금액'])))
    time_for_prepro=time_for_prepro.drop(time, axis=1)

# 계산된 시간대별 총매출액을 기준으로 총매출액 비율 계산
for time_revenue in times_revenue:
    time_revenue = make_ratio(time_for_prepro, time_revenue)
    new_table_2018 = pd.merge(new_table_2018, time_revenue, right_index=True, left_index=True)

# 연령대별 총매출 비율
total_categoiesy_revenue = store_2018.groupby(['상권_코드', '기준_분기_코드', '서비스_업종_코드_명'])['분기당_매출_금액'].sum()
quarter_ratio_revenue = store_2018.groupby(['상권_코드','기준_분기_코드'])['분기당_매출_금액'].sum()
tmp_df = (total_categoiesy_revenue / quarter_ratio_revenue).unstack(level=-1)
new_table_2018 = pd.merge(new_table_2018, tmp_df, right_index=True, left_index=True)

new_table_2018.head()
new_table_2018 = new_table_2018.fillna(0)
new_table_2018 = new_table_2018.unstack()
new_table_2018.columns = ['2018년 '+str(j) +  ' 분기 ' + str(i) for i, j in new_table_2018.columns]

new_table = pd.merge(new_table, new_table_2018, right_index=True, left_index=True)
new_table

,2021년 1 분기 여성 매출 금액,2021년 2 분기 여성 매출 금액,2021년 1 분기 남성 매출 금액,2021년 2 분기 남성 매출 금액,2021년 1 분기 연령대 10 매출 금액,2021년 2 분기 연령대 10 매출 금액,2021년 1 분기 연령대 20 매출 금액,2021년 2 분기 연령대 20 매출 금액,2021년 1 분기 연령대 30 매출 금액,2021년 2 분기 연령대 30 매출 금액,2021년 1 분기 연령대 40 매출 금액,2021년 2 분기 연령대 40 매출 금액,2021년 1 분기 연령대 50 매출 금액,2021년 2 분기 연령대 50 매출 금액,2021년 1 분기 연령대 60 이상 매출 금액,2021년 2 분기 연령대 60 이상 매출 금액,2021년 1 분기 시간대 00~06 매출액,2021년 2 분기 시간대 00~06 매출액,2021년 1 분기 시간대 06~11 매출액,2021년 2 분기 시간대 06~11 매출액,2021년 1 분기 시간대 11~14 매출액,2021년 2 분기 시간대 11~14 매출액,2021년 1 분기 시간대 14~17 매출액,2021년 2 분기 시간대 14~17 매출액,2021년 1 분기 시간대 17~21 매출액,2021년 2 분기 시간대 17~21 매출액,2021년 1 분기 시간대 21~24 매출액,2021년 2 분기 시간대 21~24 매출액,2021년 1 분기 가전,2021년 2 분기 가전,2021년 1 분기 교육,2021년 2 분기 교육,2021년 1 분기 기타,2021년 2 분기 기타,2021년 1 분기 농수산물,2021년 2 분기 농수산물,2021년 1 분기 미용,2021년 2 분기 미용,2021년 1 분기 여가,2021년 2 분기 여가,...,2018년 1 분기 교육,2018년 2 분기 교육,2018년 3 분기 교육,2018년 4 분기 교육,2018년 1 분기 기타,2018년 2 분기 기타,2018년 3 분기 기타,2018년 4 분기 기타,2018년 1 분기 농수산물,2018년 2 분기 농수산물,2018년 3 분기 농수산물,2018년 4 분기 농수산물,2018년 1 분기 미용,2018년 2 분기 미용,2018년 3 분기 미용,2018년 4 분기 미용,2018년 1 분기 여가,2018년 2 분기 여가,2018년 3 분기 여가,2018년 4 분기 여가,2018년 1 분기 운송장비,2018년 2 분기 운송장비,2018년 3 분기 운송장비,2018년 4 분기 운송장비,2018년 1 분기 음식,2018년 2 분기 음식,2018년 3 분기 음식,2018년 4 분기 음식,2018년 1 분기 의료,2018년 2 분기 의료,2018년 3 분기 의료,2018년 4 분기 의료,2018년 1 분기 의류,2018년 2 분기 의류,2018년 3 분기 의류,2018년 4 분기 의류,2018년 1 분기 편의시설,2018년 2 분기 편의시설,2018년 3 분기 편의시설,2018년 4 분기 편의시설
상권_코드,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1000001,0.536839,0.550910,0.401229,0.388514,0.004629,0.008672,0.089526,0.159500,0.168035,0.179625,0.258008,0.205077,0.164013,0.223253,0.253856,0.163298,0.835285,0.556968,6.839895,6.152917,0.303365,0.325921,0.303549,0.285469,0.275303,0.278418,0.045415,0.043466,-1.000000,-1.000000,0.036231,0.025179,-1.000000,-1.000000,-1.000000,-1.000000,0.105437,0.082550,-1.000000,-1.000000,...,0.036587,0.027994,0.031062,0.020682,0.004331,0.006887,0.001138,0.001393,0.000000,0.000000,0.000000,0.000000,0.022740,0.022089,0.031938,0.021804,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.615997,0.574796,0.542944,0.723609,0.012981,0.008734,0.006975,0.003791,0.084352,0.151614,0.136298,0.075554,0.223011,0.207886,0.249645,0.153166
1000002,0.539067,0.497391,0.423041,0.478170,0.004235,0.002083,0.186656,0.136804,0.176573,0.176321,0.204849,0.225849,0.203784,0.227031,0.186011,0.207473,1.803000,1.258649,6.030559,6.193667,0.272554,0.281284,0.357113,0.342988,0.242619,0.272574,0.048048,0.037734,-1.000000,-1.000000,-1.000000,-1.000000,0.001418,0.000948,-1.000000,-1.000000,0.007136,0.001573,0.005744,0.004352,...,0.002435,0.002773,0.003695,0.011757,0.098520,0.370198,0.294225,0.192861,0.000000,0.000000,0.000000,0.000000,0.024972,0.018415,0.025761,0.028886,0.005854,0.011341,0.009519,0.021103,0.000000,0.00000,0.000000,0.000000,0.141336,0.093232,0.109170,0.123432,0.056037,0.032668,0.036229,0.026736,0.507183,0.362330,0.370433,0.440295,0.143238,0.100772,0.118185,0.129542
1000003,0.372600,0.373874,0.542733,0.531110,0.002199,0.003525,0.212092,0.239427,0.292881,0.258305,0.166224,0.156257,0.139362,0.145764,0.102576,0.101706,1.712387,1.837665,4.860777,5.180167,0.253126,0.276004,0.358296,0.289483,0.265619,0.269148,0.059849,0.095262,0.007205,0.005451,0.008727,0.011363,0.096980,0.103175,0.000030,-1.000000,0.493331,0.412665,0.000947,0.002410,...,0.035959,0.025018,0.017336,0.021063,0.265889,0.117921,0.233118,0.162164,0.007655,0.008981,0.019107,0.011958,0.197284,0.147664,0.208432,0.260118,0.001186,0.000905,0.001075,0.001535,0.000000,0.00000,0.000000,0.000000,0.367445,0.585555,0.364236,0.421564,0.003394,0.003678,0.004579,0.006364,0.000626,0.000000,0.000000,0.000000,0.106170,0.102286,0.144271,0.104649
1000004,0.411807,0.366163,0.577116,0.620050,0.010928,0.010321,0.353791,0.322687,0.123618,0.165043,0.139339,0.140690,0.211079,0.222165,0.150169,0.125308,3.630118,3.591929,9.270843,9.135487,0.253818,0.263913,0.198561,0.218451,0.308543,0.271482,0.116830,0.123938,0.005453,0

In [104]:
full_table = pd.merge(left=store_code, right=new_table, on='상권_코드', how='left')
print(full_table.shape)
full_table.head(5)

(1490, 252)


,상권_코드,상권_구분_코드_명,2021년 1 분기 여성 매출 금액,2021년 2 분기 여성 매출 금액,2021년 1 분기 남성 매출 금액,2021년 2 분기 남성 매출 금액,2021년 1 분기 연령대 10 매출 금액,2021년 2 분기 연령대 10 매출 금액,2021년 1 분기 연령대 20 매출 금액,2021년 2 분기 연령대 20 매출 금액,2021년 1 분기 연령대 30 매출 금액,2021년 2 분기 연령대 30 매출 금액,2021년 1 분기 연령대 40 매출 금액,2021년 2 분기 연령대 40 매출 금액,2021년 1 분기 연령대 50 매출 금액,2021년 2 분기 연령대 50 매출 금액,2021년 1 분기 연령대 60 이상 매출 금액,2021년 2 분기 연령대 60 이상 매출 금액,2021년 1 분기 시간대 00~06 매출액,2021년 2 분기 시간대 00~06 매출액,2021년 1 분기 시간대 06~11 매출액,2021년 2 분기 시간대 06~11 매출액,2021년 1 분기 시간대 11~14 매출액,2021년 2 분기 시간대 11~14 매출액,2021년 1 분기 시간대 14~17 매출액,2021년 2 분기 시간대 14~17 매출액,2021년 1 분기 시간대 17~21 매출액,2021년 2 분기 시간대 17~21 매출액,2021년 1 분기 시간대 21~24 매출액,2021년 2 분기 시간대 21~24 매출액,2021년 1 분기 가전,2021년 2 분기 가전,2021년 1 분기 교육,2021년 2 분기 교육,2021년 1 분기 기타,2021년 2 분기 기타,2021년 1 분기 농수산물,2021년 2 분기 농수산물,2021년 1 분기 미용,2021년 2 분기 미용,...,2018년 1 분기 교육,2018년 2 분기 교육,2018년 3 분기 교육,2018년 4 분기 교육,2018년 1 분기 기타,2018년 2 분기 기타,2018년 3 분기 기타,2018년 4 분기 기타,2018년 1 분기 농수산물,2018년 2 분기 농수산물,2018년 3 분기 농수산물,2018년 4 분기 농수산물,2018년 1 분기 미용,2018년 2 분기 미용,2018년 3 분기 미용,2018년 4 분기 미용,2018년 1 분기 여가,2018년 2 분기 여가,2018년 3 분기 여가,2018년 4 분기 여가,2018년 1 분기 운송장비,2018년 2 분기 운송장비,2018년 3 분기 운송장비,2018년 4 분기 운송장비,2018년 1 분기 음식,2018년 2 분기 음식,2018년 3 분기 음식,2018년 4 분기 음식,2018년 1 분기 의료,2018년 2 분기 의료,2018년 3 분기 의료,2018년 4 분기 의료,2018년 1 분기 의류,2018년 2 분기 의류,2018년 3 분기 의류,2018년 4 분기 의류,2018년 1 분기 편의시설,2018년 2 분기 편의시설,2018년 3 분기 편의시설,2018년 4 분기 편의시설
0,1000001,골목상권,0.536839,0.550910,0.401229,0.388514,0.004629,0.008672,0.089526,0.159500,0.168035,0.179625,0.258008,0.205077,0.164013,0.223253,0.253856,0.163298,0.835285,0.556968,6.839895,6.152917,0.303365,0.325921,0.303549,0.285469,0.275303,0.278418,0.045415,0.043466,-1.000000,-1.000000,0.036231,0.025179,-1.000000,-1.000000,-1.000000,-1.000000,0.105437,0.082550,...,0.036587,0.027994,0.031062,0.020682,0.004331,0.006887,0.001138,0.001393,0.000000,0.000000,0.000000,0.000000,0.022740,0.022089,0.031938,0.021804,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.0,0.0,0.615997,0.574796,0.542944,0.723609,0.012981,0.008734,0.006975,0.003791,0.084352,0.151614,0.136298,0.075554,0.223011,0.207886,0.249645,0.153166
1,1000002,골목상권,0.539067,0.497391,0.423041,0.478170,0.004235,0.002083,0.186656,0.136804,0.176573,0.176321,0.204849,0.225849,0.203784,0.227031,0.186011,0.207473,1.803000,1.258649,6.030559,6.193667,0.272554,0.281284,0.357113,0.342988,0.242619,0.272574,0.048048,0.037734,-1.000000,-1.000000,-1.000000,-1.000000,0.001418,0.000948,-1.000000,-1.000000,0.007136,0.001573,...,0.002435,0.002773,0.003695,0.011757,0.098520,0.370198,0.294225,0.192861,0.000000,0.000000,0.000000,0.000000,0.024972,0.018415,0.025761,0.028886,0.005854,0.011341,0.009519,0.021103,0.000000,0.00000,0.0,0.0,0.141336,0.093232,0.109170,0.123432,0.056037,0.032668,0.036229,0.026736,0.507183,0.362330,0.370433,0.440295,0.143238,0.100772,0.118185,0.129542
2,1000003,골목상권,0.372600,0.373874,0.542733,0.531110,0.002199,0.003525,0.212092,0.239427,0.292881,0.258305,0.166224,0.156257,0.139362,0.145764,0.102576,0.101706,1.712387,1.837665,4.860777,5.180167,0.253126,0.276004,0.358296,0.289483,0.265619,0.269148,0.059849,0.095262,0.007205,0.005451,0.008727,0.011363,0.096980,0.103175,0.000030,-1.000000,0.493331,0.412665,...,0.035959,0.025018,0.017336,0.021063,0.265889,0.117921,0.233118,0.162164,0.007655,0.008981,0.019107,0.011958,0.197284,0.147664,0.208432,0.260118,0.001186,0.000905,0.001075,0.001535,0.000000,0.00000,0.0,0.0,0.367445,0.585555,0.364236,0.421564,0.003394,0.003678,0.004579,0.006364,0.000626,0.000000,0.000000,0.000000,0.106170,0.102286,0.144271,0.104649
3,1000004,골목상권,0.411807,0.366163,0.577116,0.620050,0.010928,0.010321,0.353791,0.322687,0.123618,0.165043,0.139339,0.140690,0.211079,0.222165,0.150169,0.125308,3.630118,3.591929,9.270843,9.135487,0.253818,0.263913,0.198561,0.218451,0.308543,0.271482,0.116830,0.123938,0.005453,0.100524,-1.000000,-1.000000,-1.000000,-1.000000,0.015544,0.016927,-1.000000,-1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000517,0.000477,0.000897,0.0

In [142]:
full_table.fillna(-1, inplace=True)
print(full_table.isnull().sum().sum())

0


In [143]:
def making_coeff(index):
    lr = LinearRegression()
    lr.fit(np.arange(len(full_table.iloc[index,2:].values)).reshape(-1, 1), full_table.iloc[index,2:].values)
    full_table.loc[index,'lr_coeff'] = lr.coef_

In [144]:
for i in range(full_table.shape[0]):
    making_coeff(i)
full_table.head()

,상권_코드,상권_구분_코드_명,2021년 1 분기 여성 매출 금액,2021년 2 분기 여성 매출 금액,2021년 1 분기 남성 매출 금액,2021년 2 분기 남성 매출 금액,2021년 1 분기 연령대 10 매출 금액,2021년 2 분기 연령대 10 매출 금액,2021년 1 분기 연령대 20 매출 금액,2021년 2 분기 연령대 20 매출 금액,2021년 1 분기 연령대 30 매출 금액,2021년 2 분기 연령대 30 매출 금액,2021년 1 분기 연령대 40 매출 금액,2021년 2 분기 연령대 40 매출 금액,2021년 1 분기 연령대 50 매출 금액,2021년 2 분기 연령대 50 매출 금액,2021년 1 분기 연령대 60 이상 매출 금액,2021년 2 분기 연령대 60 이상 매출 금액,2021년 1 분기 시간대 00~06 매출액,2021년 2 분기 시간대 00~06 매출액,2021년 1 분기 시간대 06~11 매출액,2021년 2 분기 시간대 06~11 매출액,2021년 1 분기 시간대 11~14 매출액,2021년 2 분기 시간대 11~14 매출액,2021년 1 분기 시간대 14~17 매출액,2021년 2 분기 시간대 14~17 매출액,2021년 1 분기 시간대 17~21 매출액,2021년 2 분기 시간대 17~21 매출액,2021년 1 분기 시간대 21~24 매출액,2021년 2 분기 시간대 21~24 매출액,2021년 1 분기 가전,2021년 2 분기 가전,2021년 1 분기 교육,2021년 2 분기 교육,2021년 1 분기 기타,2021년 2 분기 기타,2021년 1 분기 농수산물,2021년 2 분기 농수산물,2021년 1 분기 미용,2021년 2 분기 미용,...,2018년 2 분기 교육,2018년 3 분기 교육,2018년 4 분기 교육,2018년 1 분기 기타,2018년 2 분기 기타,2018년 3 분기 기타,2018년 4 분기 기타,2018년 1 분기 농수산물,2018년 2 분기 농수산물,2018년 3 분기 농수산물,2018년 4 분기 농수산물,2018년 1 분기 미용,2018년 2 분기 미용,2018년 3 분기 미용,2018년 4 분기 미용,2018년 1 분기 여가,2018년 2 분기 여가,2018년 3 분기 여가,2018년 4 분기 여가,2018년 1 분기 운송장비,2018년 2 분기 운송장비,2018년 3 분기 운송장비,2018년 4 분기 운송장비,2018년 1 분기 음식,2018년 2 분기 음식,2018년 3 분기 음식,2018년 4 분기 음식,2018년 1 분기 의료,2018년 2 분기 의료,2018년 3 분기 의료,2018년 4 분기 의료,2018년 1 분기 의류,2018년 2 분기 의류,2018년 3 분기 의류,2018년 4 분기 의류,2018년 1 분기 편의시설,2018년 2 분기 편의시설,2018년 3 분기 편의시설,2018년 4 분기 편의시설,lr_coeff
0,1000001,골목상권,0.536839,0.550910,0.401229,0.388514,0.004629,0.008672,0.089526,0.159500,0.168035,0.179625,0.258008,0.205077,0.164013,0.223253,0.253856,0.163298,0.835285,0.556968,6.839895,6.152917,0.303365,0.325921,0.303549,0.285469,0.275303,0.278418,0.045415,0.043466,-1.000000,-1.000000,0.036231,0.025179,-1.000000,-1.000000,-1.000000,-1.000000,0.105437,0.082550,...,0.027994,0.031062,0.020682,0.004331,0.006887,0.001138,0.001393,0.000000,0.000000,0.000000,0.000000,0.022740,0.022089,0.031938,0.021804,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.0,0.0,0.615997,0.574796,0.542944,0.723609,0.012981,0.008734,0.006975,0.003791,0.084352,0.151614,0.136298,0.075554,0.223011,0.207886,0.249645,0.153166,-0.001751
1,1000002,골목상권,0.539067,0.497391,0.423041,0.478170,0.004235,0.002083,0.186656,0.136804,0.176573,0.176321,0.204849,0.225849,0.203784,0.227031,0.186011,0.207473,1.803000,1.258649,6.030559,6.193667,0.272554,0.281284,0.357113,0.342988,0.242619,0.272574,0.048048,0.037734,-1.000000,-1.000000,-1.000000,-1.000000,0.001418,0.000948,-1.000000,-1.000000,0.007136,0.001573,...,0.002773,0.003695,0.011757,0.098520,0.370198,0.294225,0.192861,0.000000,0.000000,0.000000,0.000000,0.024972,0.018415,0.025761,0.028886,0.005854,0.011341,0.009519,0.021103,0.000000,0.00000,0.0,0.0,0.141336,0.093232,0.109170,0.123432,0.056037,0.032668,0.036229,0.026736,0.507183,0.362330,0.370433,0.440295,0.143238,0.100772,0.118185,0.129542,-0.002538
2,1000003,골목상권,0.372600,0.373874,0.542733,0.531110,0.002199,0.003525,0.212092,0.239427,0.292881,0.258305,0.166224,0.156257,0.139362,0.145764,0.102576,0.101706,1.712387,1.837665,4.860777,5.180167,0.253126,0.276004,0.358296,0.289483,0.265619,0.269148,0.059849,0.095262,0.007205,0.005451,0.008727,0.011363,0.096980,0.103175,0.000030,-1.000000,0.493331,0.412665,...,0.025018,0.017336,0.021063,0.265889,0.117921,0.233118,0.162164,0.007655,0.008981,0.019107,0.011958,0.197284,0.147664,0.208432,0.260118,0.001186,0.000905,0.001075,0.001535,0.000000,0.00000,0.0,0.0,0.367445,0.585555,0.364236,0.421564,0.003394,0.003678,0.004579,0.006364,0.000626,0.000000,0.000000,0.000000,0.106170,0.102286,0.144271,0.104649,-0.002545
3,1000004,골목상권,0.411807,0.366163,0.577116,0.620050,0.010928,0.010321,0.353791,0.322687,0.123618,0.165043,0.139339,0.140690,0.211079,0.222165,0.150169,0.125308,3.630118,3.591929,9.270843,9.135487,0.253818,0.263913,0.198561,0.218451,0.308543,0.271482,0.116830,0.123938,0.005453,0.100524,-1.000000,-1.000000,-1.000000,-1.000000,0.015544,0.016927,-1.000000,-1.000000,...,0.000000,0.000000,0.000000,0.000517,0.000477,0.000897,0.000322,0.031